In [10]:
import geopandas
import numpy as np
import pandas as pd
import xarray as xr
from scipy.stats import lognorm,cumfreq
from scipy.spatial import cKDTree
from src.calculate_ds import calc_dwi



### Load Building data


In [53]:

## load buildings
df_build_fac = geopandas.read_file('DATA/bld_sc/bldPort_TV0_siteclasses.shp')


### Calculating the weighted damage states ($DS_i$) for TV0<sub>pga</sub> as a result of $N_e'$  earthquakes




In [55]:
DSwi=[]
for build in df_build_fac.index.values:
######### obtaining fragility function and defining the paramaters needed
    if df_build_fac.iloc[build]['vulnStrEQ']=='BrM+LC+LR':
        tb='T1'
    elif df_build_fac.iloc[build]['vulnStrEQ']=='BrCfl+LC+LR':
        tb='T2' 
    elif df_build_fac.iloc[build]['vulnStrEQ']=='BrCri+LC+LR':
        tb='T3' 
    dwi=calc_dwi(tb,build)
    DSwi.append(dwi)
    
DSwi=np.array(DSwi)

### Calculating the weighted damage states ($DS_i$) after policy intervention as a result of $N_e'$  earthquakes


In [51]:
policies=[1,2,3,4,5] # policies
eqs=20 # number of earthquakes
ds=4 # damage states
DSwi_px=np.zeros((len(policies),len(df_build_fac.index.values),eqs,ds)) # array containing damage states after policy intervention
cp=np.zeros(5) ## number of retrofits (or cp)

for ip,p in enumerate(policies):
    nb=0 ## initialise the variable to store number of retrofits
    for build in df_build_fac.index.values:
        if p==1:
            if df_build_fac.iloc[build]['vulnStrEQ']=='BrM+LC+LR':
                tb='T1_upg'
                nb+=1
            elif df_build_fac.iloc[build]['vulnStrEQ']=='BrCfl+LC+LR':
                tb='T2' 
            elif df_build_fac.iloc[build]['vulnStrEQ']=='BrCri+LC+LR':
                tb='T3' 
            
        elif p==3:
            if df_build_fac.iloc[build]['vulnStrEQ']=='BrM+LC+LR':
                if df_build_fac.iloc[build]['cluster']==2:
                    tb='T1_upg'
                    nb+=1
                else:
                    tb='T1'
            elif df_build_fac.iloc[build]['vulnStrEQ']=='BrCfl+LC+LR':
                tb='T2' 
            elif df_build_fac.iloc[build]['vulnStrEQ']=='BrCri+LC+LR':
                tb='T3' 

        elif p==2:
            if df_build_fac.iloc[build]['vulnStrEQ']=='BrM+LC+LR':
                if df_build_fac.iloc[build]['polytype']=='lowIncome':
                    tb='T1_upg'
                    nb+=1
                else:
                    tb='T1'
            elif df_build_fac.iloc[build]['vulnStrEQ']=='BrCfl+LC+LR':
                tb='T2' 
            elif df_build_fac.iloc[build]['vulnStrEQ']=='BrCri+LC+LR':
                tb='T3' 

        elif p==4:
            if df_build_fac.iloc[build]['vulnStrEQ']=='BrM+LC+LR':
                if df_build_fac.iloc[build]['cluster']==2:
                    tb='T1_upg'
                    nb+=1
                else:
                    tb='T1'
            elif df_build_fac.iloc[build]['vulnStrEQ']=='BrCfl+LC+LR':

                if df_build_fac.iloc[build]['cluster']==2:
                    tb='T2_upg'
                    nb+=1
                else:
                    tb='T2'
            elif df_build_fac.iloc[build]['vulnStrEQ']=='BrCri+LC+LR':
                tb='T3' 


        else:
            if df_build_fac.iloc[build]['vulnStrEQ']=='BrM+LC+LR':
                if df_build_fac.iloc[build]['cluster']==2:
                    tb='T1_upg'
                    nb+=1
                
                else:
                    tb='T1'
       
            elif df_build_fac.iloc[build]['vulnStrEQ']=='BrCfl+LC+LR':

                if df_build_fac.iloc[build]['cluster']==2:
                    tb='T2_upg'
                    nb+=1
                else:
                    tb='T2'

            elif df_build_fac.iloc[build]['vulnStrEQ']=='BrCri+LC+LR':
                tb='T3' 

        dwi=calc_dwi(tb,build)
        DSwi_px[ip,build]=np.array(dwi)
        
    cp[ip]=nb


### Calculating $D_i, D_{low}$ and reference values for Cost-Benefit Analysis (CBA)

In [77]:
DSwi_nop=np.sum(DSwi,axis=2)
di_eq_nop=np.mean(DSwi_nop,axis=1)/np.max(np.mean(DSwi_nop,axis=1))
sort_di = np.sort(di_eq_nop)
ecdf_di = np.arange(1, len(sort_di) + 1) / len(sort_di)
area_ecdf_nop=np.trapz(y=sort_ia,x=ecdf_di)

sort_di_li=np.sort(di_eq_nop[df_build_fac['polytype']=='lowIncome'])
ecdf_di_li = np.arange(1, len(sort_di_li) + 1) / len(sort_di_li)
area_ecdf_li_nop=np.trapz(y=sort_di_li,x=ecdf_di_li)



### Calculating $D_i, D_{low} \Delta_{all}, \Delta_{low}, BCR_{all}$ and $BCR_{low}$ for policy interventions

In [ ]:
#initialise CBA arrays
benefit1=np.zeros(5)
benefit2=np.zeros(5)
costbenefit1=np.zeros(5)
costbenefit2=np.zeros(5)

In [80]:
####change policy number
p=1

####for entire tv0
di=np.mean(np.sum(DSwi_px[p-1],axis=2),axis=1)/np.max(np.mean(DSwi_nop,axis=1))
sort_di = np.sort(di)
ecdf_di = np.arange(1, len(sort_di) + 1) / len(sort_di)
area_ecdf_p=np.trapz(sort_di,ecdf_di)


#########for low income
sort_di_li=np.sort(di[df_build_fac['polytype']=='lowIncome'])
ecdf_di_li = np.arange(1, len(sort_di_li) + 1) / len(sort_di_li)
area_ecdf_li_p=np.trapz(y=sort_di_li,x=ecdf_di_li)

########benefit calculation
benefit1[p-1]=area_ecdf_nop-area_ecdf_p
benefit2[p-1]=area_ecdf_li_nop-area_ecdf_li_p
print(f'For Policy No. {p}, Benefit 1 is {benefit1[p-1]} and Benefit 2 is {benefit2[p-1]}')

######### BCR (Benefit-Cost ratio) calculation


costbenefit1[p-1]=benefit1[p-1]/cp[p-1]
costbenefit2[p-1]=benefit2[p-1]/cp[p-1]
print(f'BCR1 is {costbenefit1[p-1]} and BCR2 is {costbenefit2[p-1]}')

For Policy No. 1, Benefit 1 is 0.11196514629798365 and Benefit 2 is 0.1329594417464391
BCR1 is 5.86204954439705e-05 and BCR2 is 6.961227316567492e-05


### Write the outputs for plotting  (Only run if you want to re-store any missing output files -or- update analysis results)


In [83]:
####change policy number
for p in [1,2,3,4,5]:

    ####for entire tv0
    di=np.mean(np.sum(DSwi_px[p-1],axis=2),axis=1)/np.max(np.mean(DSwi_nop,axis=1))
    sort_di = np.sort(di)
    ecdf_di = np.arange(1, len(sort_di) + 1) / len(sort_di)
    area_ecdf_p=np.trapz(sort_di,ecdf_di)


    #########for low income
    sort_di_li=np.sort(di[df_build_fac['polytype']=='lowIncome'])
    ecdf_di_li = np.arange(1, len(sort_di_li) + 1) / len(sort_di_li)
    area_ecdf_li_p=np.trapz(y=sort_di_li,x=ecdf_di_li)

    ########benefit calculation
    benefit1[p-1]=area_ecdf_nop-area_ecdf_p
    benefit2[p-1]=area_ecdf_li_nop-area_ecdf_li_p
    print(f'For Policy No. {p}, Benefit 1 is {benefit1[p-1]} and Benefit 2 is {benefit2[p-1]}')

    ##############

    ######### BCR (Benefit-Cost ratio) calculation


    costbenefit1[p-1]=benefit1[p-1]/cp[p-1]
    costbenefit2[p-1]=benefit2[p-1]/cp[p-1]
    print(f'BCR1 is {costbenefit1[p-1]} and BCR2 is {costbenefit2[p-1]}')
    np.save(f'output_files/Di_policy{p}',di)

For Policy No. 1, Benefit 1 is 0.11196514629798365 and Benefit 2 is 0.1329594417464391
BCR1 is 5.86204954439705e-05 and BCR2 is 6.961227316567492e-05
For Policy No. 2, Benefit 1 is 0.11134266976315355 and Benefit 2 is 0.1329594417464391
BCR1 is 6.52653398377219e-05 and BCR2 is 7.793636679158211e-05
For Policy No. 3, Benefit 1 is 0.08954048972293044 and Benefit 2 is 0.10692444821547992
BCR1 is 8.487250210704307e-05 and BCR2 is 0.00010135018788197148
For Policy No. 4, Benefit 1 is 0.09638228595795112 and Benefit 2 is 0.11509455415856806
BCR1 is 8.287384863108437e-05 and BCR2 is 9.896350314580229e-05
For Policy No. 5, Benefit 1 is 0.09638228595795112 and Benefit 2 is 0.11509455415856806
BCR1 is 6.797058248092462e-05 and BCR2 is 8.116682239673348e-05


In [85]:
np.save(f'output_files/Di_nop',di_eq_nop)
np.save(f'output_files/delta_all',benefit1)
np.save(f'output_files/delta_low',benefit2)
np.save(f'output_files/bcr_all',costbenefit1)
np.save(f'output_files/bcr_low',costbenefit2)

